In [1]:
from dataloader import *
from model import *
import matplotlib.pyplot as plt

In [9]:
X_train = np.load("data\X_train.npy")
X_val = np.load("data\X_validation.npy")
X_test = np.load("data\X_test.npy")

y_train = np.load("data\y_train.npy")
y_val = np.load("data\y_validation.npy")
y_test = np.load("data\y_test.npy")

In [13]:
dm = DelightDataModule(X_train=X_train, 
                       X_val=X_val, 
                       X_test=X_test, 
                       y_train=y_train,
                       y_val=y_val, 
                       y_test=y_test, 
                       batch_size=4, 
                       num_workers=0, 
                       seed=0, 
                       train_augmentation="delight")
dm.setup()

In [ ]:
idx = 3

batch = next(iter(dm.train_dataloader()))
imgs = batch[0]
targets = batch[1]


plt.imshow(imgs[idx][0][0])
plt.scatter(targets[idx][0] +15, targets[idx][1]+15, marker="*", s =100, color="black")

plt.show()

In [ ]:
class Delight(L.LightningModule):

    def __init__(self, config):
        super(Delight, self).__init__()

        self.bottleneck = torch.nn.Sequential(
            torch.nn.Conv2d(config["channels"], config["nconv1"], 3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(config["nconv1"], config["nconv2"], 3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(config["nconv2"], config["nconv3"], 3),
            torch.nn.ReLU(),
            torch.nn.Flatten(),
        )

        self.regression = torch.nn.Sequential(
            torch.nn.Linear(
                in_features=4 * 4 * config["nconv3"] * config["levels"],
                out_features=config["ndense"],
            ),
            torch.nn.Tanh(),
            torch.nn.Dropout(p=config["dropout"]),
            torch.nn.Linear(in_features=config["ndense"], out_features=2),
        )

        self.loss = torch.nn.MSELoss()

        self.lr = config["lr"]
        self.weight_decay = config["weight_decay"]

        self.training_predictions = []
        self.training_classes = []

        self.val_predictions = []
        self.val_classes = []

        self.curves = {
            "train_loss": [],
            "val_loss": [],
        }

        #self.save_files = config["save_files"]

        self.config = config
        self.save_hyperparameters(config)

    # def initialize_weights(self):
    #     for m in self.modules():
    #         if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
    #             torch.nn.init.xavier_uniform_(m.weight) #Glorot Uniform
    #             torch.nn.init.constant_(m.bias, 0.1)

    def forward(self, x):

        original_shape = x.shape
        new_shape = original_shape[:-4] + (-1,)

        leading = torch.prod(
            torch.tensor(original_shape[:-3])
        ).item()  # Batch*Transforms*Levels

        x = x.reshape(
            leading, original_shape[-3], original_shape[-2], original_shape[-1]
        )

        x = self.bottleneck(x)
        x = x.reshape(*new_shape)
        x = self.regression(x)

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        x_hat = self.forward(x)
        train_loss = self.loss(x_hat, y)

        self.training_predictions.append(x_hat.detach().cpu())
        self.training_classes.append(y.cpu())

        self.log("train_loss", train_loss, prog_bar=True)
        return train_loss

    def validation_step(self, batch, batch_idx):

        x, y = batch
        x_hat = self.forward(x)
        val_loss = self.loss(x_hat, y)

        self.val_predictions.append(x_hat.cpu())
        self.val_classes.append(y.cpu())

        self.log("val_loss", val_loss, prog_bar=True)

        return val_loss

    def on_train_epoch_end(self):

        predictions = torch.cat(self.training_predictions, dim=0)
        classes = torch.cat(self.training_classes, dim=0)

        self.curves["train_loss"].append(self.loss(predictions, classes))

        self.training_predictions.clear()
        self.training_classes.clear()

    def on_validation_epoch_end(self):

        predictions = torch.cat(self.val_predictions, dim=0)
        classes = torch.cat(self.val_classes, dim=0)

        self.curves["val_loss"].append(self.loss(predictions, classes))

        self.val_predictions.clear()
        self.val_classes.clear()

    def predict_step(self, batch):
        x, _ = batch
        return self(x)

    def configure_optimizers(self):
        # lr_lambda = lambda epoch: 0.95 ** epoch

        optimizer = torch.optim.Adam(
            self.parameters(), lr=self.lr, weight_decay=self.weight_decay
        )
        # lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lr_lambda])
        # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=40, eta_min=0, last_epoch=-1)
        return [optimizer]  # , [scheduler]

In [7]:
config = {
    "nconv1": 52,
    "nconv2": 57,
    "nconv3": 41,
    "ndense": 685,
    "dropout": 0.06,
    "channels": 1,
    "levels": 5,
    "lr": 0.1,
    "weight_decay": 0.1
}

model = Delight(config)

In [16]:
out = model(imgs)

torch.Size([160, 656])
torch.Size([4, 8, 3280])


In [18]:
out.shape

torch.Size([4, 8, 2])

In [19]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parámetros entrenables: {trainable_params}")

Parámetros entrenables: 2297184


In [ ]:
            torch.nn.Linear(
                in_features=4 * 4 * config["nconv3"] * config["levels"],
                out_features=config["ndense"],
            ),

In [15]:
imgs.shape

torch.Size([4, 8, 5, 1, 30, 30])